# Objetivo a ser alcançado 

###O objetivos são métricas de análise baseadas nas conclusões de treinos e nas métricas obtidas com o objetivo de avaliar seus resultados finais, e tomar decições com base nos valores finais a pos o pipeline.

#### **Inclução das bibliotecas utilizadas** ####


*   Nesta enfase iremos inserir os recursos de utilização.
*   Base de carregamentos e perifericos ajusteveis.
*   algoritmos em potencial.
*   Criação da listagem de todos os pipelines metamorficos do modelo.



* Carregamento do conjunto do texto principal ele sera nosso principal menu de classes de dados de treinamento

In [ ]:
from sklearn.datasets import fetch_20newsgroups 

 > Carregamento das bibliotecas de aprendizado de máquina

In [ ]:
from sklearn.datasets import fetch_20newsgroups  
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer  
from sklearn.naive_bayes import MultinomialNB 
from sklearn.pipeline import Pipeline  
from sklearn.pipeline import Pipeline  
from sklearn.linear_model import SGDClassifier  
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay


>  Carregando bibliotecas de manipulação e análise de dados





In [ ]:
import pandas as pd 
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import nltk 
import matplotlib.pyplot as plt 

Ignorando warnings de alerta


> 1.   [Iremos desativar os avisos, pois iremos nós concentrarmos apenas nós experimentos, leia mais sobre o porquê desta decisão ]( https://docs.python.org/pt-br/dev/library/warnings.html )
2. [O que e isso ?](https://www.google.com/search?q=para+que+serve+warning&sxsrf=APwXEdcPhCZybk1nYjLvp4dydkfuaMNRqQ%3A1682036584144&ei=aNdBZK7CCLfy5OUPwbK6sA8&oq=o+que+significa+warnings&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQARgAMgoIABBHENYEELADMgoIABBHENYEELADMgoIABBHENYEELADMgoIABBHENYEELADMgoIABBHENYEELADMgoIABBHENYEELADMgoIABBHENYEELADMgoIABBHENYEELADSgQIQRgAUABYAGDCDmgDcAB4AIABAIgBAJIBAJgBAMgBCMABAQ&sclient=gws-wiz-serp)


In [ ]:
import warnings
warnings.simplefilter('ignore')

#### **Carregamento da base de dados principal**####
> Transportar do conjunto de treino de classes

In [ ]:
newsgroups = fetch_20newsgroups(subset='train') 

Exibindo conjunto de dados de texto recem obtidas



In [ ]:
list(newsgroups.target_names)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

**Seleção do conjunto de textos que iremos utilizar.**
> A nossa base de classes está bastante ondulante, pois nosso objetivo e medir a relevância de citação desses assuntos em particular, a escolha sera tendenciosa a nosso resultado.



In [ ]:
categories = ['sci.space','sci.electronics','alt.atheism','sci.med',
              'rec.autos','comp.graphics','comp.os.ms-windows.misc',
              'comp.sys.ibm.pc.hardware','sci.crypt','comp.sys.mac.hardware']
df_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
df_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True)

######Listagem do conjunto de classes pre-selecionadas para estudo######

In [ ]:
df_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'rec.autos',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space']

###Feature Engineering##

 > Obtendo algoritmo de conversão, este algoritmo e de suma importância é um método para converter texto em dados numéricos.

In [ ]:
count_vect = CountVectorizer()


Iremos definir o[ TF-IDF](https://pt.wikipedia.org/wiki/Tf%E2%80%93idf#:~:text=O%20valor%20tf%E2%80%93idf%20(abrevia%C3%A7%C3%A3o,ou%20em%20um%20corpus%20lingu%C3%ADstico.) é uma medida estatística que tem o intuito de indicar a importância de uma palavra de um documento em relação a uma coleção de documentos ou em um corpus linguístico

In [ ]:
X_train_counts = count_vect.fit_transform(df_train.data)
X_train_counts.shape

(5790, 92542)

##Estudo de caso **1**

> #### **Treinamento do modelo - 1**

Iremos definir nosso pipeline seu comportamento sera reunir as várias etapas do processo, que podem ser validadas de forma cruzada ao definir parâmetros diferentes.


In [ ]:
clf_2 = Pipeline([
    ('vect', CountVectorizer()), 
    ('tfidf', TfidfTransformer()),  
    ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=25, random_state=42))]) # Passo 3, aplicar o algoritmo Naive Bayes

> Treinando o modelo e realizando predições dos dados

In [ ]:
svm_trained = clf_2.fit(df_train.data, df_train.target)  
pred = svm_trained.predict(df_test.data)
acc = np.mean(pred == df_test.target)

###Emitindo status da acurácia

In [ ]:
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8562532433834976


### **Tuning de parametros**

#####Modelo utilizado : **Stochastic Gradient Descent** em conjunto com **GridSearchCV**

In [ ]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}
            
gs_clf_svm = GridSearchCV(svm_trained, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(df_train.data, df_train.target)

#### Capturando os melhores parametros

In [ ]:
print(gs_clf_svm.best_score_)
gs_clf_svm.best_params_

0.9148531951640759


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

#### Realizando a predição dos dados de teste

In [ ]:
pred = gs_clf_svm.predict(df_test.data)
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8489880643487286


##### A recall é a razão em que é o número de verdadeiros positivos e o número de falsos negativos, tendo isso como parâmetro principal, vimos que a relação entre as demais classes apesar de ondulante não está tão distante a, sim, podemos concluir uma assertividade moderado, pois  seus números não estão destoantemente distantes podemos nós assegurar pelos menos 80%

In [ ]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.82      0.94      0.88       319
           comp.graphics       0.86      0.80      0.83       389
 comp.os.ms-windows.misc       0.82      0.81      0.81       394
comp.sys.ibm.pc.hardware       0.79      0.71      0.75       392
   comp.sys.mac.hardware       0.85      0.85      0.85       385
               rec.autos       0.91      0.96      0.94       396
               sci.crypt       0.82      0.96      0.89       396
         sci.electronics       0.86      0.66      0.75       393
                 sci.med       0.90      0.85      0.88       396
               sci.space       0.85      0.96      0.90       394

                accuracy                           0.85      3854
               macro avg       0.85      0.85      0.85      3854
            weighted avg       0.85      0.85      0.85      3854



####Iremos dividir as nossas N abordagens para diferenças nossos diferentes resultados de análise.

####**A bordagem 1**

##### Incluido parametros de determinância 

*   Incluido algoritmo naive bayes
*   Filtrando nossas operções


In [ ]:
svm_stp_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),  
    ('tfidf', TfidfTransformer()),  
    ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=25, random_state=42))]) 

> Realizando o treinamento dos dados e a predição do conjunto de teste

In [ ]:
svm_stp_trained = svm_stp_clf.fit(df_train.data, df_train.target)
pred = svm_stp_trained.predict(df_test.data) 

In [ ]:
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8601453035806954


Obtendo status de assertividade

In [ ]:
svm_stp_tun = GridSearchCV(svm_stp_trained, parameters_svm, n_jobs=-1)
svm_stp_tun = svm_stp_tun.fit(df_train.data, df_train.target)
print(svm_stp_tun.best_score_)
svm_stp_tun.best_params_

0.9162348877374784


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

> Obtendo classificação de assertividade 

In [ ]:
pred = svm_stp_tun.predict(df_test.data)
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8523611831862999


> ##### Nesta situação nossos conjuntos de retorno estão ainda mais centrados, iremos nós concentrar nos valores próximos, chegamos a conclusão que há um certo volume em grandezas,  iremos continuar a definir nossas abordagens.

In [ ]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.84      0.94      0.89       319
           comp.graphics       0.85      0.80      0.82       389
 comp.os.ms-windows.misc       0.82      0.83      0.83       394
comp.sys.ibm.pc.hardware       0.77      0.71      0.74       392
   comp.sys.mac.hardware       0.86      0.86      0.86       385
               rec.autos       0.89      0.95      0.92       396
               sci.crypt       0.84      0.96      0.90       396
         sci.electronics       0.85      0.68      0.76       393
                 sci.med       0.91      0.86      0.88       396
               sci.space       0.87      0.95      0.91       394

                accuracy                           0.85      3854
               macro avg       0.85      0.85      0.85      3854
            weighted avg       0.85      0.85      0.85      3854



####**A bordagem 2**

> Aplicando filtros de ação e inserindo algoritmos de transformação e acertividade e treinando o modelo

In [ ]:
nb_clf_stp = Pipeline([
    ('vect', CountVectorizer(stop_words='english')), 
    ('tfidf', TfidfTransformer()),  
    ('clf', MultinomialNB())])   

nb_clf_stp_trained = nb_clf_stp.fit(df_train.data, df_train.target)  

> Realizando a predição dos dos nossos conjuntos de simulação inseridos.


In [ ]:
pred = nb_clf_stp_trained.predict(df_test.data)
acc = np.mean(pred == df_test.target)
print('Accuracy = ', acc)

creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

Accuracy =  0.8500259470679813
                          precision    recall  f1-score   support

             alt.atheism       0.97      0.90      0.94       319
           comp.graphics       0.84      0.78      0.81       389
 comp.os.ms-windows.misc       0.85      0.73      0.78       394
comp.sys.ibm.pc.hardware       0.72      0.82      0.77       392
   comp.sys.mac.hardware       0.87      0.84      0.86       385
               rec.autos       0.92      0.97      0.94       396
               sci.crypt       0.76      0.97      0.85       396
         sci.electronics       0.84      0.67      0.75       393
                 sci.med       0.94      0.87      0.90       396
               sci.space       0.86      0.95      0.91       394

                accuracy                           0.85      3854
               macro avg       0.86      0.85      0.85      3854
            weighted avg       0.86      0.85      0.85      3854



##Estudo de caso **2**

#### **Feature Enginnering 2** 

######Transformando o conjunto de texto em uma matriz de numérica de tokens, lembrando que este passo e de suma importância, pois e obrigação nossa deixar os dados o mais sutil possível.
 

In [ ]:
tfidf_transformer = TfidfTransformer() 
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

> #### **Treinamento do modelo - 2**

 ##### Obtendo algoritmo naive bayes para utilização e treinamento dos conjuntos.

In [ ]:
clf = MultinomialNB()
clf.fit(X_train_tfidf, df_train.target)

MultinomialNB()

> Aplicando filtros de tratamento

In [ ]:
clf_1 = Pipeline([
    ('vect', CountVectorizer()),     
    ('tfidf', TfidfTransformer()),  
    ('clf', MultinomialNB())])

> Treinando dados utilzando o nosso conjunto recem transformado como numerico e realizando a predição dos dado de teste.

In [ ]:
clf_trained = clf_1.fit(df_train.data, df_train.target) 
pred = clf_trained.predict(df_test.data)

In [ ]:
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.816554229372081


> Posição final de métricas sobre assertividade do modelo treinado.

In [ ]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.97      0.88      0.92       319
           comp.graphics       0.85      0.70      0.77       389
 comp.os.ms-windows.misc       0.87      0.67      0.76       394
comp.sys.ibm.pc.hardware       0.72      0.78      0.75       392
   comp.sys.mac.hardware       0.88      0.79      0.83       385
               rec.autos       0.88      0.97      0.92       396
               sci.crypt       0.58      0.97      0.73       396
         sci.electronics       0.85      0.62      0.71       393
                 sci.med       0.93      0.87      0.90       396
               sci.space       0.87      0.93      0.90       394

                accuracy                           0.82      3854
               macro avg       0.84      0.82      0.82      3854
            weighted avg       0.84      0.82      0.82      3854



#### **Tuning de parametros 2**


> Iremos definir nossos parametros de listagem e constancia de parâmetros a seguir estaremos realizando a busca pelo melhor parametro e treinamento do modelo.

In [ ]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(clf_trained, parameters, n_jobs=-1) # Define o grid search para buscar os melhores parametros 
gs_clf = gs_clf.fit(df_train.data, df_train.target)

> Nossa maior grandesa dos valores

In [ ]:
print(gs_clf.best_score_)
gs_clf.best_params_

0.9202072538860104


{'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

> Aplicando a predição e obtendo os status de qualidade dessas predições.

In [ ]:
pred = gs_clf.predict(df_test.data) 
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8448365334717177


> Nosso diagnóstico final mostra uma boa abordagem dos nossos dados, isso demonstra que nossa visando nosso  número de resultados verdadeiros positivos, está ótimo para nosso objetivo alcançado. 

In [ ]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.95      0.94      0.94       319
           comp.graphics       0.76      0.81      0.78       389
 comp.os.ms-windows.misc       0.79      0.65      0.71       394
comp.sys.ibm.pc.hardware       0.69      0.76      0.72       392
   comp.sys.mac.hardware       0.84      0.84      0.84       385
               rec.autos       0.94      0.95      0.94       396
               sci.crypt       0.87      0.94      0.90       396
         sci.electronics       0.80      0.78      0.79       393
                 sci.med       0.91      0.88      0.90       396
               sci.space       0.92      0.92      0.92       394

                accuracy                           0.84      3854
               macro avg       0.85      0.85      0.85      3854
            weighted avg       0.85      0.84      0.84      3854



####**A bordagem 1**

> Carregando processos de dados

In [ ]:
clf_2 = Pipeline([
    ('vect', CountVectorizer()),  
    ('tfidf', TfidfTransformer()), 
    ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=25, random_state=42))]) # Passo 3, aplicar o algoritmo Naive Bayes

Realizando o treinamento dos dados visando seus novos parametros e obtendo a predição dos novos conjuntos recem trienados em seguida retornada nosso status de acuracio dos valores.

In [ ]:
svm_trained = clf_2.fit(df_train.data, df_train.target)
pred = svm_trained.predict(df_test.data)
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8562532433834976


##### **Tuning de parametros**


In [ ]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}
gs_clf_svm = GridSearchCV(svm_trained, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(df_train.data, df_train.target)

Realizando a visualização dos melhores parametros da assertividade

In [ ]:
print(gs_clf_svm.best_score_)
gs_clf_svm.best_params_

0.9148531951640759


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

 ##### Recorrendo a predição dos dados

In [ ]:
pred = gs_clf_svm.predict(df_test.data)
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8489880643487286


Analise final 

In [ ]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.82      0.94      0.88       319
           comp.graphics       0.86      0.80      0.83       389
 comp.os.ms-windows.misc       0.82      0.81      0.81       394
comp.sys.ibm.pc.hardware       0.79      0.71      0.75       392
   comp.sys.mac.hardware       0.85      0.85      0.85       385
               rec.autos       0.91      0.96      0.94       396
               sci.crypt       0.82      0.96      0.89       396
         sci.electronics       0.86      0.66      0.75       393
                 sci.med       0.90      0.85      0.88       396
               sci.space       0.85      0.96      0.90       394

                accuracy                           0.85      3854
               macro avg       0.85      0.85      0.85      3854
            weighted avg       0.85      0.85      0.85      3854



####**A bordagem 2**

#### Aplicando parametros de filtros

In [ ]:
clf_1 = Pipeline([
    ('vect', CountVectorizer()),    
    ('tfidf', TfidfTransformer()),   
    ('clf', MultinomialNB())])       

> ##### Realizando a predição dos dados recem avaliados e executando o treinamento do modelo

In [ ]:
clf_trained = clf_1.fit(df_train.data, df_train.target) 
pred = clf_trained.predict(df_test.data) 
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.816554229372081


> Gerando metricas finais para nossa analise de assertividade do modelo

In [ ]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.97      0.88      0.92       319
           comp.graphics       0.85      0.70      0.77       389
 comp.os.ms-windows.misc       0.87      0.67      0.76       394
comp.sys.ibm.pc.hardware       0.72      0.78      0.75       392
   comp.sys.mac.hardware       0.88      0.79      0.83       385
               rec.autos       0.88      0.97      0.92       396
               sci.crypt       0.58      0.97      0.73       396
         sci.electronics       0.85      0.62      0.71       393
                 sci.med       0.93      0.87      0.90       396
               sci.space       0.87      0.93      0.90       394

                accuracy                           0.82      3854
               macro avg       0.84      0.82      0.82      3854
            weighted avg       0.84      0.82      0.82      3854

